In [17]:
import sys
import os
sys.path.append(os.path.abspath(".."))
print(sys.path)

import os
print(os.listdir("../scripts"))


import ee
from scripts.aoi_loader import load_shapefile


# Inizializza GEE
ee.Authenticate()
ee.Initialize()

# 1. Carica AOI
aoi = load_shapefile("../data/aoi_4326.shp")


""" # 3. Visualizza in mappa con geemap (opzionale)
import geemap
Map = geemap.Map()
Map.addLayer(img, {"min": -25, "max": 0}, "SAR VV")
Map """


['C:\\Users\\Minutella Francesco\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\Minutella Francesco\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'C:\\Users\\Minutella Francesco\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'C:\\Users\\Minutella Francesco\\AppData\\Local\\Programs\\Python\\Python312', 'c:\\Users\\Minutella Francesco\\jk\\SAR\\venv', '', 'c:\\Users\\Minutella Francesco\\jk\\SAR\\venv\\Lib\\site-packages', 'c:\\Users\\Minutella Francesco\\jk\\SAR\\venv\\Lib\\site-packages\\win32', 'c:\\Users\\Minutella Francesco\\jk\\SAR\\venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\Minutella Francesco\\jk\\SAR\\venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\Minutella Francesco\\jk\\SAR', 'c:\\Users\\Minutella Francesco\\jk\\SAR', 'c:\\Users\\Minutella Francesco\\jk\\SAR', 'c:\\Users\\Minutella Francesco\\jk\\SAR', 'c:\\Users\\Minutella Francesco\\jk\\SAR', 'c:\\Users\\Minutella Francesco\\jk\\SAR', 'c:\\Users\\Minutella Francesco\\jk\\SAR

ModuleNotFoundError: No module named 'scripts.aoi_loader'